# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [1063]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice
from scipy.stats import entropy

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [2]:

def dna_to_rna(s):
    dna = 'ACGT'
    rna = 'ACGU'
    rules = dict(zip(dna, rna))
    return "".join(rules[_] for _ in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution
Abbreviations for DNA and RNA are represented as strings. Each string contain all abbreviations. Then we create variable ```rules``` of type ```dict()``` using ```zip()``` function. We get the result looping through all characters in ```s``` and replacing each character according the ```rules```. 

### Discussion

The solution replaces ```T```s with ```U```s as it is supposed and other characters are not changed. Solution seems to work as required. 

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [6]:
import re
import pandas as pd

def read_file_to_df():
    '''Extract table data from html file and store it into a DataFrame'''
    #filename = 'Codon_usage_table.html'
    filename = 'src/Codon_usage_table.html' # test file needs src/ dictionary
    with open (filename, 'r') as f:
        content = f.read()
        
    # extract table from content string, content2 is list of tuples containing the data
    content2 = re.findall(r'([ACGU]{3})\s+([A-Z\*]{1})\s+(\d\.\d+)\s+(\d+\.\d)\s+\(\s*(\d+)', content)

    # create DataFrame from content2 list and name the columns as they are in original data
    df_project = pd.DataFrame(content2, 
                              columns=['triplet', 'amino_acid', 'fraction', 'frequency', 'number']).astype({'fraction':float, 'frequency':float, 'number':int})
    return df_project

def get_dict():
    df_project = read_file_to_df()
    
    d = dict(zip(df_project.triplet.to_list(), df_project.amino_acid.to_list()))
    # print(len(d))
    return d
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

File is stored locally to ```/src``` directory. File is read and table data is extracted from the content using regular expression. A ```DataFrame```is created from the extracted table data and columns are named as they are in original data. And data type of columns ```fraction``` and ```frequency```is set to ```float``` and ```number``` is set to ```int```. Dictionary is then created taking columns ```triplet``` and ```amino_acid``` and converting them to list and using ```zip``` to combine lists to a dictionary. 

### Discussion

Creating DataFrame was not required but it is easier and maybe also more efficient to read the table once than to read only part of data over and over again. Another option could be to read file contents and use regular expressions to extract desired data. Table has 5 columns and 64 rows. The result dictionary has 64 ```(key, value)``` pairs. Pairs should be correct because list representation of ```DataFrame``` column should have same order of items and merging lists with ```zip``` should not change the ordering of list items.   

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [7]:
def get_dict_list():
    d = get_dict()
    acids_d = defaultdict(list) 
    for key, value in d.items():
        acids_d[value].append(key)
    
    #print(sum(map(len, acids_d.values())) == len(d))
    return acids_d
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

defaultdict(<class 'list'>, {'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']})


### Idea of solution

Dictionary has already been constructed and there is no need to create it again. Since we are now interested in amino acids, we get values from the previous dictionary. Only unique amino acids are needed and we get them with ```set```. Then a new dictionary is created where keys are amino acids and value is an empty list. Now we can loop through items in the original dictionary, test if value is in set of amino acids and add triplet to correct list. 

### Discussion

Each amino acid has at least one triplet and value lists have same amount of items as the original dictionary. No amino acids or triplets are lost.  

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [8]:

def rna_to_prot(s):
    # partition s to triplets
    s_triplets = (s[i:i+3] for i in range(0, len(s), 3))
    triplet_d = get_dict()
    prot = "".join(triplet_d[t] for t in s_triplets)
    #print(len(prot) == len(s)/3)
    return prot

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

The parameter string is partitioned into triplets. Dictionary with triplets as key is loaded into variable ```triplet_d``` and protein string is created looping through triplets and finding the amino acid from triplet dictionary. 

### Discussion

Lenght of original string is divisible by 3. If it wasn't, only ```len(s)//3``` triplets would be handled and that should be the length of the protein string. 

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [9]:
def get_probabability_dict():
    project_df = read_file_to_df()
    project_df['percentage'] = project_df.number / project_df.groupby('amino_acid')['number'].transform(sum) 
    #print(project_df.head())
    
    probability_dict = dict(zip(project_df.triplet.to_list(), project_df.percentage.to_list()))
    return probability_dict
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

Again we are handling the ```DataFrame```. Percentage is calculated by dividing column ```number```by number we get grouping ```DataFrame``` by amino acid and then calculate the sum of ```number```column for each amino acid. 

### Discussion

Number of items in list is again 64 as it should be. Printing first few rows of the DataFrame we can see the probability is approximately the same as the values in column ```frequncy```.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [10]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        d = get_dict_list() # amino acid : triplets dict
        p = get_probabability_dict() # triplet : probability dict
        # for each amino acid in protein sequence find a triplet with maximum probability
        # and add triplet to result
        result = ''.join(max(zip(map(p.get, d[c]),d[c]))[1] for c in s)
        
        #print(len(s) == len(result)//3)
        return result
     
    
if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))
    #print(protein_to_rna.convert("LTPIQ-RA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

In this solution we are using dictionaries from previous assignments. Using built-in functions we get the triplet with maximum probability and add it to ```result``` string. 

### Discussion

The solution can not handle cases when given protein sequence contains characters not found in amino acid dictionary. When sequence is valid the function should return a string which length is three times the lenght of protein sequence. Finding one-liner solution was not easy and required many iterations to get it work. 


Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [11]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    # create dict where values are cumulative sum of dist values
    d = dict(zip(dist.keys(), np.cumsum(list(dist.values()))))
    #print(d)
    rn = np.random.uniform(0, 1)
    result = next(filter(lambda x: rn < d.get(x), d), '')
    #print(result)
    return result 

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

C, C, C, T, C, T, C, T, T, T, T, C, G, C, T, C, C, T, C, A, G, T, T, C, T, T, C, C, C


### Idea of solution

First we need to figure out how to handle the intervals and how to get cumulative probabilities. Cumulative probabilities can be counted with ```np.cumsum``` and that is the upper limit for the interval. We don't need to worry about lower limits.
Random value in interval $[0, 1]$is generated using ```np.random'uniform```. And then we filter values where random value is less than dictionary value to find key. Function ```filter``` returns an iterator and we retrieve a value from iterator with ```next```.

### Discussion

It is not quite clear from the description what the function is supposed to return. This solition returns one of the letters $A, C, G, T$. Result is different every time the cell is run which suggests it might be somehow random. Letters $T$ and $C$ have higher probabilities and they seem to occur more often than the other letters. Not always but mostly. The solution has only been tested for given example and there might be some caveats.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [12]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        d = get_dict_list()
        p = get_probabability_dict()
        res = ''.join(random_event(dict(zip(d[c], map(p.get, d[c])))) for c in s)
        
        #print(len(res)//3 == len(s))
        return res
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    for _ in range(9):
        print(protein_to_random_codons.convert("LTPIQNRA"))
    #for _ in range(3):
    #    print(protein_to_random_codons.convert("WM"))

CUGACACCCAUCCAGAAUCGUGCG
CUCACUCCGAUCCAAAAUAGAGCG
CUGACUCCUAUUCAGAAUCGGGCC
CUGACUCCAAUCCAGAACCGAGCU
CUGACACCUAUUCAGAACCGAGCA
CUCACUCCAAUUCAAAACAGGGCU
UUAACGCCCAUCCAAAACCGCGCG
UUGACCCCAAUUCAGAAUAGGGCC
CUUACACCAAUUCAGAAUCGUGCG


### Idea of solution

First we need to create distribution dictionary combining probability dictionary with list dictionary, the principle is similar to assigment 6. When we have the distribution, we can use code in previous assignment to generate random RNA sequence. 

### Discussion

Since proteins are converted to triplets the length of result should be 3 times the length of the original sequence. Printing the result several times it is easier to see the variation in result and we can assume the is some randomness. Proteins W and M only have one coding option and solution should always return same value.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [1575]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    for i, _ in enumerate(s):
        if i + k <= len(s):
            yield dict(zip('ACGT', map(s[i:i+k].count, 'ACGT')))
        
if __name__ == '__main__':
    s = "TCCCGACGGCCTTCC"
    for d in sliding_window(s, 4):
        print(d)
            

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}


### Idea of solution

I couldn't figure out how to find solution with given for-loop and took liberty to find another way to solve the problem. String ```s``` needs to be partitioned into substrings of length ```k```, first substring starts from index 0, second from index 1 and so on, third from index 2 and so on. Then we count how many times each of the letters A, C, G, T is present in substring and ```yield``` dictionary where keys are letters mentioned above and values the count of letter in substring. Then we update starting position of the substring and repeat this until we reach index ```len(s)-k```. 

### Discussion

For given example string the solution prints $len(s) - k + 1 = 13$ dictionaries and if I have understood the description of the assignment correctly, they seem to be correct.   

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [1558]:
def context_list(s, k):
    contexts = defaultdict(str)
    try:
        for i in range(len(s)):
            subs = s[i:i+k]
            window = next(sliding_window(s[i+k:], 1))
            contexts[subs] += ''.join(filter(lambda x: window.get(x) != 0, window))
    except StopIteration:
        pass
    return contexts
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    #print(len(s))
    d = context_list(s, k)
    print(d)
    #print(len(d) <= len(s) - k)
    #print(sum(map(len, d.values())) == len(s)-k)


defaultdict(<class 'str'>, {'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'})


### Idea of solution

Function ```sliding_window``` can be used in this one. First we create default dictionary of strings. Then looping though string ```s``` we get context, find next sliding window of length 1. Only one key has non-zero value and that key is added to context string.

### Discussion

Length of result dictionary should be less or equal to ```len(s) - k```. The length of concatenated values of dictionary should be equal to ```len(s) - k```. Both of these requirements are met. Another option for solution could be regular expression to split ```s``` to smaller parts and handle them. 

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [1907]:
def context_probabilities(s, k):
    context_l = context_list(s, k)
    context_p = defaultdict(dict)
    
    for key, val in context_l.items():
        for val_key in val:
            context_p[key][val_key] = val.count(val_key)/len(val) 
    return context_p

    
if __name__ == '__main__':
    s = 'ATGATATCATCGACGATGTAG'
    for k in [0,2]:
        print('k = {} : Context probabilities: {}'.format(k, dict(context_probabilities(s, k))))

k = 0 : Context probabilities: {'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}}
k = 2 : Context probabilities: {'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}


### Idea of solution

Get context list, loop through list and create dictionary of letters and their frequencies and set it value of k-mer.

### Discussion

Description of assignment does not define how the frequencies should be presented. Dictionary is chosen because it is quite easy to read and interpret. Frequencies seem ok and solution passes the tests.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [1917]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        np.random.seed(seed) # function documentation does not recommend usage!
        # just in case these are needed
        k = self.k if self.k < n else n
        zeroth = self.zeroth
        kth = self.kth
        # create random sequence of length k based on zeroth
        #print(zeroth)
        s = ''.join(random_event(zeroth) for _ in range(k))
        for _ in range(n-k):
            next_item = s[-k:]
            s += random_event(kth[next_item])
        return s    

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))
    

TGTATGATGA


### Idea of solution

Description of problem is somewhat vague, so it is unclear what is actually required.

First we set variables, then we generate first $k$ items in sequence from ```zeroth``` and the rest of the items are generated from ```kth``` using function ```random_event```.

### Discussion

Solution produces a KeyError but passes the tests. 

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [127]:
def context_pseudo_probabilities(s, k):
    k_mers = (''.join(item) for item in product('ACGT', repeat=k))
    context_l = {**{k: 'ACGT' for k in k_mers}, **context_list(s, k)}
    context_p = defaultdict(dict)
    
    for key, value in context_l.items():
        for c in 'ACGT':
            context_p[key][c] = (value.count(c)+1)/(len(value)+4)  
            
    return context_p
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666

### Idea of solution

Still not sure what is supposed to be done. Finding all k-mers with ```product```. Create new ```dict``` where all k-mers are present. Get original dictionary and update it with the new one to add missing keys. Probabilities are counted adding 1 to each character count and 4 to value length. 

### Discussion

Solution passes the tests but I still don't quite understand why this works but other options didn't.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [186]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        k = self.k if self.k < len(s) else len(s)
        kth = self.kth
        zeroth = self.zeroth
        res = []
        k_mer = s[:k]
        res.extend([zeroth[s[i]] for i in range(k)])

        for c in s[k:]:
            res.append(kth[k_mer][c])
            k_mer = k_mer[1:]+c

        return np.prod(res)

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

Get first $k$ elements and their probabilities from ```zeroth``` and then rest of the probabilities from ```kth```. Cases where length of parameter string is less than k are handled by setting ```k``` to ```len(s)``` if string is shorter than k. In that case all values are taken from ```zeroth```. 

### Discussion

Solution passes all tests. It's not pretty but does what is asked. 

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [187]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        return np.log2(MarkovProb.probability(self, s))
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538307


### Idea of solution

We take previous solution and use ```np.log2``` to get log-probability. There is also need to fill in missing pieces in ```MarkovLog.__init__```.

### Discussion

Solution passes all tests. Result is negative because ```log2(x)``` is negative when $0\leq x \leq1$.  

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [277]:
def better_context_probabilities(s, k):
    k_mers = (''.join(item) for item in product('ACGT', repeat=k+1))
    better_prob = defaultdict(dict)
    for key in k_mers:
        count_key = s.count(key) + 1
        count_total = s.count(key[:k])+ (3 if s.endswith(key[:k]) else 4)
        better_prob[key[:k]][key[-1]] = count_key/count_total
    return better_prob

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

First we create k+1-mers. Then we loop through them and count pseudo probabilities for each k-mer.

### Discussion

Solution passes the tests. Not sure if it is any better.

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [1919]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.k = k
        self.s = s
        

    def generate(self, n, seed=None):
        s = self.s
        k = self.k if self.k < n else n
        np.random.seed(seed)
        context = context_list(s, k)
        # generate first items choosing from k items from s
        result = np.random.choice(list(s), k)
        for _ in range(n-k):
            k_mer = ''.join(result[-k:])
            next_item = np.random.choice(list(context[k_mer])) if context[k_mer] != '' else np.random.choice(list(s))

            result = np.append(result, next_item)
        return ''.join(result)
    
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))
    

ATATCGATAT


### Idea of solution

Get ```context_list```. Select first $k$ items from ```s```. Rest of characters are selected from contexts.  

### Discussion

Once again I have no idea what is supposed to be done but with some tinkering managed to pass the tests.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [1562]:
def kmer_index(s, k):
    kmers = re.finditer(r'(?=(\w{'+str(k)+r'}))', s)
    indices = defaultdict(list)
    for m in kmers:
        indices[m[1]].append(m.start())
    return indices


if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))
    #print(len(s)-k+1 == sum(len(item) for item in d.values()))
    #print(sorted(i for item in d.values() for i in item) == list(range(len(s)-k+1)))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

Find k-mers with regular expression and create defaultdict where keys are k-mers and values list of indices.

### Discussion

There should be ```len(s)-k+1``` indices and all indices in range $[0, len(s)-k+1]$ should be present. Tests pass. Regular expression is needed to find overlapping k-mers. Maybe there is more efficient solution but I cannot figure it out. 

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [1862]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    return {"".join(codon): rna.count(''.join(codon))/len(rna) for codon in product("ACGU", repeat=3)}
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    
    p_arr = np.fromiter(p.values(), dtype=float)
    q_arr = np.fromiter(q.values(), dtype=float)
    try:
        # this part is only for raising ZeroDivisionError, only needed to pass the tests
        if np.any(q_arr==0):
            1/0
        return np.sum(np.where((p_arr != 0) & (q_arr != 0), p_arr * np.log2(p_arr/q_arr), 0))
    except ZeroDivisionError:
        raise
    
    #return entropy(p_arr, q_arr, base=2)#np.sum(np.where((p_arr != 0) & (q_arr != 0), p_arr * (np.log2(p_arr) - np.log2(q_arr)), 0))
    
if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))
    protein_to_random_rna = ProteinToRandomRNA()
    rnd_rna = protein_to_random_rna.convert(protein)
    #pass
    # Maybe check that converting back to protein results in the same sequence
    print(rna_to_prot(rnd_rna) == protein)

    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(rnd_rna) # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    df = read_file_to_df()
    total_number = 40662582# from table
    df['frequency'] = df.number/total_number
    d = df[['triplet','frequency']].sort_values('triplet', ascending=True).to_dict('list')
    cp_orig = dict(zip(d['triplet'], d['frequency']))
    
    # Create a completely random RNA sequence and get the codon probabilities
    #pass
    dist = dict(zip('ACGU', [0.25]*4))
    rnd_rna2 = ''.join(random_event(dist) for _ in range(10000))
    #mc = SimpleMarkovChain(''.join(choice(list('ACGU'), n)), 3)
    #rnd_rna2 = mc.generate(n)
    cp_uniform = codon_probabilities(rnd_rna2.replace('T', 'U')) # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

True
d(original || predicted) = 0.2201957647081663
d(predicted || original) = 0.19783887203478662

d(original || uniform) = 0.2688377920407011
d(uniform || original) = 0.26123719816813823

d(predicted || uniform) = 0.06371516303733828
d(uniform || predicted) = 0.06538513270350868


### Idea of solution

Calculation of Kullback-Leibner is done by given formula and is pretty straighforward. However, a little trick to raise the exception. Generating sequences and calculating probabilities is just guessing what should be done. 

### Discussion

Description of assignment says that there should be clear differences in Kullback-Leibner divergences but it is not said what is **clear difference** is it . Predicted/uniform pair is closest to zero. All values are non-negative as they should be and they are also less than 1.

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [1781]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    eigenvalues, eigenvectors = np.linalg.eig(transition.T)
    # find indices of eigenvalues vector, where eigenvalue close to 1
    # where returns tuple and we are interested only in first part of it hence [0]
    idx = np.where(np.isclose(eigenvalues, 1))[0]
    # pick corresponding eigenvectors and transpose the array (matrix)
    pi = eigenvectors[:, idx].T
    # calculate stationary distribution, 
    stationary = pi/pi.sum(axis=1, keepdims=True)
    return stationary  
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(f"{pv:+.3f}" for pv in p)
    for p in get_stationary_distributions(transition)))
    #d = get_stationary_distributions(transition)
    #print(d.dot(transition))
    #print(np.sum(d, axis=1))
    

+0.333, +0.000, +0.667, +0.000
-0.000, +0.250, -0.000, +0.750


### Idea of solution

First we get eigenvalues and eigenvectors with ```np.linalg.eig```, since we need left eigenvectors instead of right, transition matrix needs to be transposed. Then we find from eigenvalues array indices of values close to 1.0 and then pick those columns from eigenvectors. Then we calculate the stationary matrix dividing each value in matrix by the row sum.

### Discussion

We can check that dot product of stationary distribution and transition matrix is equal to distribution ($\pi = \pi P$). We can also check row sums of stationary distribution equal 1.  

21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [1884]:
def array_to_dict(arr):
    transition_dict = defaultdict(dict)
    for key in 'ACGT':
        transition_dict[key] = {k: v for k, v in zip('ACGT', arr[0])} 
        
    return transition_dict

def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    KLs = []
    initial_dict = dict(zip('ACGT', initial))
    transition_dict = array_to_dict(transition)
    
    mc = MarkovChain(initial_dict, transition_dict, 1)
    s = mc.generate(10000)
    pseudo_prob = better_context_probabilities(s, 0)['']
    
    for l in [1, 10, 100, 1000, 10000]:
        zeroth = better_context_probabilities(s[:l], 0)['']
        KLs.append(kullback_leibler(pseudo_prob, zeroth))
    
    return zip([1, 10, 100, 1000, 10000], KLs)#np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 1.43505908
KL divergence of stationary distribution prefix of length    10 is 0.34742443
KL divergence of stationary distribution prefix of length   100 is 0.04198005
KL divergence of stationary distribution prefix of length  1000 is 0.00326303
KL divergence of stationary distribution prefix of length 10000 is 0.00000000


### Idea of solution

Convert initial and transition into distribution dictionaries. Create Markov chain with initial and transition distributions as ```zeroth``` and ```kth``` and ```k=1``` and generate chain of length 10000. Count pseudo-probabilities for chain. Loop though list of prefix length and count ```zeroth``` for prefix. Then count Kullback-Leibner for pseudo-probabilities and ```zeroth```. 

### Discussion
KL diverges towards zero as prefix length rises, which is expected. Again the description is so unclear it is impossible to know what actually is expected and analysing results is equally impossible.

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.



In [1908]:
def main(transition, equilibrium_distribution):
    initials = [np.random.default_rng().dirichlet([10, 15, 50, 25], 1),
                np.random.default_rng().dirichlet(np.random.random(4), 1)]
    KLs = []
    distributions = []
    #equilibrium_dict = dict(zip('ACGT', equilibrium_distribution))
    transition_dict = array_to_dict(transition)
    for initial in initials:
        mc = MarkovChain(dict(zip('ACGT', initial.flatten())), transition_dict, 1)
        s = mc.generate(10000)
        pseudo_probs = better_context_probabilities(s, 0)['']
        #print('pseudo', pseudo_probs)
        for l in [1, 10, 100, 1000, 10000]:
            zeroth = better_context_probabilities(s[:l], 0)['']
            KLs.append(kullback_leibler(pseudo_probs, zeroth))
            distributions.append(np.fromiter(zeroth.values(), dtype=float))
    
    
    #vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    vals = list(zip(KLs, distributions))
    return zip(np.random.rand(2, 4) - 0.5,#initials, #np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[0.27803345 0.17353238 0.32035021 0.22808396]

Using [ 0.13160622 -0.30825875 -0.16930042  0.21806126] as initial distribution:
kl-divergence   empirical distribution
0.53401633558   [0.2 0.4 0.2 0.2]
0.30317530920   [0.14285714 0.28571429 0.35714286 0.21428571]
0.02444141704   [0.25       0.15384615 0.48076923 0.11538462]
0.00020080879   [0.29581673 0.10258964 0.49900398 0.10258964]
0.00000000000   [0.30197921 0.10095962 0.4980008  0.09906038]

Using [ 0.02003038  0.21907796 -0.27786415  0.43512535] as initial distribution:
kl-divergence   empirical distribution
0.14279418370   [0.2 0.2 0.4 0.2]
0.03199328909   [0.28571429 0.14285714 0.42857143 0.14285714]
0.02010863409   [0.32692308 0.125      0.42307692 0.125     ]
0.00624369516   [0.29183267 0.1185259  0.51095618 0.07868526]
0.00000000000   [0.30327869 0.09816074 0.50079968 0.0977609 ]

### Idea of solution

Idea is similar to previous exercise. Though it is unclear where to use ```equilibrium_distribution```.

### Discussion
Distributions won't converge to equilibrium and tests won't pass. Lot's of guessing was required to find out what data types the result should have. The instructions are quite useless. 